# Multiple Results


In this notebook we show how you can in a single request, have the LLM model return multiple results per prompt. This is useful for running experiments where you want to evaluate the robustness of your prompt and the parameters of your config against a particular large language model.


In [1]:
!pip install semantic-kernel==0.9.2b1

In [2]:
from services import Service

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = Service.AzureOpenAI

In [7]:
import semantic_kernel as sk
from semantic_kernel.contents.chat_history import ChatHistory

if selectedService == Service.OpenAI or selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.open_ai_prompt_execution_settings import (
        OpenAITextPromptExecutionSettings,
        OpenAIChatPromptExecutionSettings,
    )
    from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
        AzureChatPromptExecutionSettings,
    )
    from semantic_kernel.connectors.ai.open_ai import (
        AzureTextCompletion,
        AzureChatCompletion,
        OpenAITextCompletion,
        OpenAIChatCompletion,
    )
if selectedService == Service.HuggingFace:
    from semantic_kernel.connectors.ai.hugging_face import HuggingFaceTextCompletion

First, we will set up the text and chat services we will be submitting prompts to.


In [24]:
kernel = sk.Kernel()

# Configure Azure LLM service
if selectedService == Service.AzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    azure_text_service = AzureTextCompletion(
        service_id="aoai_text", deployment_name=deployment, endpoint=endpoint, api_key=api_key
    )  # set the deployment name to the value of your text model (e.g. gpt-35-turbo-instruct or text-davinci-003)
    azure_chat_service = AzureChatCompletion(
        service_id="aoai_chat", deployment_name=deployment, endpoint=endpoint, api_key=api_key
    )  # set the deployment name to the value of your chat model

# Configure OpenAI service
if selectedService == Service.OpenAI:
    api_key, org_id = sk.openai_settings_from_dot_env()
    oai_text_service = OpenAITextCompletion(
        service_id="oai_text", ai_model_id="gpt-3.5-turbo-instruct", api_key=api_key, org_id=org_id
    )
    oai_chat_service = OpenAIChatCompletion(
        service_id="oai_chat", ai_model_id="gpt-3.5-turbo", api_key=api_key, org_id=org_id
    )

# Configure Hugging Face service
if selectedService == Service.HuggingFace:
    hf_text_service = HuggingFaceTextCompletion(service_id="hf_text", ai_model_id="distilgpt2", task="text-generation")

Next, we'll set up the completion request settings for text completion services.


In [19]:
oai_text_prompt_execution_settings = AzureChatPromptExecutionSettings(
    service="oai_text",
    extension_data={
        "max_tokens": 80,
        "temperature": 0.7,
        "top_p": 1,
        "frequency_penalty": 0.5,
        "presence_penalty": 0.5,
        "number_of_responses": 3,
    },
)

## Multiple Open AI Text Completions


In [28]:
if selectedService == Service.OpenAI:
    chat = ChatHistory()
    chat.add_user_message("What is the purpose of a rubber duck?")
    results = await oai_text_service.complete(chat_history=chat, settings=oai_text_prompt_execution_settings)
    i = 1
    for result in results:
        print(f"Result {i}: {result}")
        i += 1

## Multiple Azure Open AI Text Completions


In [36]:
if selectedService == Service.AzureOpenAI:
    chat = ChatHistory()
    chat.add_user_message("provide me a list of possible meanings for the acronym 'ORLD'")
    results = await azure_chat_service.complete_chat(chat_history=chat, settings=oai_text_prompt_execution_settings)
    i = 1
    for result in results:
        print(f"Result {i}: {result}")
        i += 1

Result 1: 1. Online Research and Learning Database
2. Operations Research and Logistics Development
3. Organization for Rural Livelihood Development
4. Occupational Rehabilitation and Disability Support
5. Optical Remote-sensing and Land Development
6. Organic Recycling and Waste Disposal
7. Open-source Robotics Learning Platform
8. Oilfield Remediation and Land Reclamation 
9. Outer Rim Logistics Division
Result 2: 1. Online Research and Learning Database
2. Open Resource Library Directory
3. Organization for Research and Learning Development
4. Operational Risk and Loss Database
5. Online Retail Logistics Division
6. Office of Regulatory and Legal Affairs
7. Orthopedic Rehabilitation and Long-term Disability
8. Overarching Research and Leadership Development
Result 3: 1. Online Research and Learning Database
2. Organization of Real-Life Development
3. Operation and Resource Logistics Division
4. Office for Research and Library Development
5. Open-source Robotics and Lab Design
6. Onl

## Multiple Hugging Face Text Completions


In [ ]:
if selectedService == Service.HuggingFace:
    from semantic_kernel.connectors.ai.hugging_face.hf_prompt_execution_settings import (
        HuggingFacePromptExecutionSettings,
    )

    hf_prompt_execution_settings = HuggingFacePromptExecutionSettings(
        service_id="hf_text", extension_data={"max_new_tokens": 80, "temperature": 0.7, "top_p": 1}
    )

In [ ]:
if selectedService == Service.HuggingFace:
    prompt = "The purpose of a rubber duck is"
    chat = ChatHistory()
    chat.add_user_message(prompt)
    results = await hf_text_service.complete(chat_history=chat, prompt_execution_settings=hf_prompt_execution_settings)
    print("".join(results))

Here, we're setting up the settings for Chat completions.


In [32]:
oai_chat_prompt_execution_settings = OpenAIChatPromptExecutionSettings(
    service_id="oai_chat",
    max_tokens=80,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
    number_of_responses=3,
)

## Multiple OpenAI Chat Completions


In [33]:
if selectedService == Service.OpenAI:
    chat = ChatHistory()
    chat.add_user_message(
        "It's a beautiful day outside, birds are singing, flowers are blooming. On days like these, kids like you..."
    )
    results = await oai_chat_service.complete_chat(chat_history=chat, settings=oai_chat_prompt_execution_settings)
    i = 0
    for result in results:
        print(f"Result {i+1}: {str(result)}")
        i += 1

## Multiple Azure OpenAI Chat Completions


In [34]:
az_oai_prompt_execution_settings = AzureChatPromptExecutionSettings(
    service_id="aoai_chat",
    max_tokens=80,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
    number_of_responses=3,
)

In [35]:
if selectedService == Service.AzureOpenAI:
    content = "Tomorow is going to be a great day, I can feel it. I'm going to wake up early, go for a run, and then..."
    chat = ChatHistory()
    chat.add_user_message(content)
    results = await azure_chat_service.complete_chat(chat_history=chat, settings=az_oai_prompt_execution_settings)
    i = 0
    for result in results:
        print(f"Result {i+1}: {str(result)}")
        i += 1

Result 1: I'm going to start my day with a healthy breakfast to fuel my body and mind. After that, I'll spend some time meditating and setting positive intentions for the day ahead. 

Next, I'll tackle my to-do list with enthusiasm and focus. Whether it's work-related tasks, personal projects, or errands, I'll approach each task with determination and a positive attitude.

During my
Result 2: I will have a productive day at work. I have prepared my to-do list and I'm determined to accomplish all my tasks efficiently. After work, I plan to meet up with some friends for a fun evening out. We might try out a new restaurant or catch a movie together. Overall, I'm excited for tomorrow and the positive energy it brings.
Result 3: I'll have a healthy breakfast to fuel my body for the day ahead. After that, I plan to spend some time meditating and setting positive intentions for the day. Once I feel centered and focused, I'll dive into my work or studies with enthusiasm and determination.

Dur

## Streaming Multiple Results

Here is an example pattern if you want to stream your multiple results. Note that this is not supported for Hugging Face text completions at this time.


In [ ]:
if selectedService == Service.OpenAI:
    import os
    from IPython.display import clear_output
    import time

    # Determine the clear command based on OS
    clear_command = "cls" if os.name == "nt" else "clear"

    chat = ChatHistory()
    chat.add_user_message("what is the purpose of a rubber duck?")

    stream = oai_text_service.complete_stream(chat_history=chat, settings=oai_text_prompt_execution_settings)
    number_of_responses = oai_text_prompt_execution_settings.number_of_responses
    texts = [""] * number_of_responses

    last_clear_time = time.time()
    clear_interval = 0.5  # seconds

    # Note: there are some quirks with displaying the output, which sometimes flashes and disappears.
    # This could be influenced by a few factors specific to Jupyter notebooks and asynchronous processing.
    # The following code attempts to buffer the results to avoid the output flashing on/off the screen.

    async for results in stream:
        current_time = time.time()

        # Update texts with new results
        for idx, result in enumerate(results):
            if idx < number_of_responses:
                texts[idx] += str(result)

        # Clear and display output at intervals
        if current_time - last_clear_time > clear_interval:
            clear_output(wait=True)
            for idx, text in enumerate(texts):
                print(f"Result {idx + 1}: {text}")
            last_clear_time = current_time

    print("----------------------------------------")